# DataJoint Element for Motion Sequencing with Keypoint MoSeq


#### **Open-source Data Pipeline for Motion Sequencing in Neurophysiology**


Welcome to the tutorial for the DataJoint Element for motion sequencing analysis. This tutorial aims to provide a comprehensive understanding of the open-source data pipeline by `element-moseq`.


In [ ]:
![pipeline](../images/flowchart.svg)

In [ ]:
import os

if os.path.basename(os.getcwd()) == "notebooks":
    os.chdir("..")

In [ ]:
import datajoint as dj
from pathlib import Path
import numpy as np


In [ ]:
dj.list_schemas()

In [ ]:
dj.schema("vathes-team_devlab_mila_kpmstest_kpms_model").drop()
dj.schema("vathes-team_devlab_mila_kpmstest_kpms_pca")

In [ ]:
from tutorial_pipeline import lab, subject, session, kpms_pca, kpms_model

## 1. FIT PCA MODEL AND LATENT DIMENSIONS


In [ ]:
(
    dj.Diagram(subject.Subject)
    + dj.Diagram(session.Session)
    + dj.Diagram(kpms_pca)
    + dj.Diagram(kpms_model)
)

In [ ]:
kpms_pca.KeypointSet()

In [ ]:
# Subject and Session tables
subject.Subject.insert1(
    dict(
        subject="subject1",
        sex="F",
        subject_birth_date="2020-01-01",
        subject_description="test",
    ),
    skip_duplicates=True,
)

# Definition of the dictionary named "session_keys"
session_keys = [
    dict(subject="subject1", session_datetime="2021-06-02 14:04:22"),
    dict(subject="subject1", session_datetime="2021-06-03 14:43:10"),
]

# Insert this dictionary in the Session table
session.Session.insert(session_keys, skip_duplicates=True)

In [ ]:
kpms_pca.PoseEstimationMethod()

In [ ]:
kpset_key = dict(subject="subject1", session_datetime="2021-06-02 14:04:22", kpset_id=1)

kpms_pca.KeypointSet.insert1(
    {
        **kpset_key,
        "format_method": "deeplabcut",
        "kpset_config_dir": "/Users/milagros/Documents/datajoint-elements/element-moseq/data/inbox/input_data",
        "kpset_videos_dir": "/Users/milagros/Documents/datajoint-elements/element-moseq/data/inbox/input_data/videos",
        "kpset_desc": "testing kpms pca schema",
    },
    skip_duplicates=True,
)

In [ ]:
kpms_pca.KeypointSet()

In [ ]:
video_files = [
    "/Users/milagros/Documents/datajoint-elements/element-moseq/data/inbox/input_data/videos/21_11_8_one_mouse.top.ir.Mp4",
    "/Users/milagros/Documents/datajoint-elements/element-moseq/data/inbox/input_data/videos/21_12_2_def6a_1.top.ir.mp4",
    "/Users/milagros/Documents/datajoint-elements/element-moseq/data/inbox/input_data/videos/21_12_2_def6b_2.top.ir.mp4",
]

kpms_pca.KeypointSet.VideoFile.insert(
    (
        {**kpset_key, "video_id": v_idx, "video_path": Path(f)}
        for v_idx, f in enumerate(video_files)
    ),
    skip_duplicates=True,
)

In [ ]:
kpms_pca.KeypointSet.VideoFile()

In [ ]:
kpms_pca.RecordingInfo()

In [ ]:
kpms_pca.RecordingInfo.populate()

In [ ]:
kpms_pca.RecordingInfo()

In [ ]:
kpms_pca.Bodyparts()

In [ ]:
bodypart_key = {**kpset_key, "bodyparts_id": 1}
kpms_pca.Bodyparts.insert1(
    {
        **bodypart_key,
        "anterior_bodyparts": ["nose"],
        "posterior_bodyparts": ["spine4"],
        "use_bodyparts": [
            "spine4",
            "spine3",
            "spine2",
            "spine1",
            "head",
            "nose",
            "right ear",
            "left ear",
        ],
    },
    skip_duplicates=True,
)

In [ ]:
kpms_pca.Bodyparts()

In [ ]:
kpms_pca.PCATask()

In [ ]:
kpms_pca.PCATask.insert1(
    {
        **bodypart_key,
        "output_dir": "/Users/milagros/Documents/datajoint-elements/element-moseq/data/outbox/kpms_project_testing",
        "task_mode": "trigger",
    },
    skip_duplicates=True,
)

In [ ]:
kpms_pca.PCATask()

In [ ]:
key = (kpms_pca.PCATask & "task_mode = 'trigger'").fetch1("KEY")
key

In [ ]:
kpms_pca.FormattedDataset()

In [ ]:
kpms_pca.FormattedDataset.populate()

In [ ]:
kpms_pca.FormattedDataset()

The `PCAFitting` computation will fit a PCA model to aligned and centered keypoint coordinates


In [ ]:
kpms_pca.PCAFitting()

In [ ]:
kpms_pca.PCAFitting.populate()

In [ ]:
kpms_pca.PCAFitting()

In [ ]:
kpms_pca.DimsExplainedVariance()

In [ ]:
kpms_pca.DimsExplainedVariance.populate()

In [ ]:
kpms_pca.DimsExplainedVariance()

In [ ]:
# Plotting before the user's choice of the latent dimensions to use in the next step
from keypoint_moseq import load_pca, plot_scree, plot_pcs
from element_moseq.readers.kpms_reader import load_dj_config

output_dir = (kpms_pca.PCATask & key).fetch1("output_dir")
config = load_dj_config(output_dir, check_if_valid=False, build_indexes=False)

pca = load_pca(output_dir)

plot_scree(pca, project_dir=output_dir)
plot_pcs(pca, project_dir=output_dir, **config)

## 2. KPMS Model prefitting and full fitting


1. **Initialization**: Auto-regressive (AR) parameters and syllable sequences are randomly initialized using pose trajectories from PCA
2. **Fitting an AR-HMM**: AR parameters, transition probabilities and syllable sequences are iteratively updated through Gibbs sampling
3. **Fitting the full model**: all params, including both AR-HMM as well as centroid, heading, noise-estimates and continuous latent states (i.e. pose trajectories) are iteratively updated through Gibss sampling. Step useful for noisy data.
4. **Extracting model results**: the learned states of the model are parsed and saved to disk for visualization and downstream analysis.


In [ ]:
(
    dj.Diagram(subject.Subject)
    + dj.Diagram(session.Session)
    + dj.Diagram(kpms_pca)
    + dj.Diagram(kpms_model)
)

Adjust `kappa` hyperparameter to achieve the desired distribution of syllable durations. Higher values of kappa lead to longer syllables.

Let's chose a kappa value that yields a median syllable duration of 12 frames.


In [ ]:
fps = kpms_pca.RecordingInfo.fetch1("fps_average")
kappa_min = (12 / fps) * 1000 #ms
kappa_max = 1e6 #ms 
kappa_range = np.logspace(np.log10(kappa_min), np.log10(kappa_max), num=3)
print('kappa range = {} ms'.format(kappa_range))


For rodents it's recommended a target duration of ~400ms


The kappa value in the config is only used during model initialization.


In [ ]:
prefitting_key = {
    **key,
    'pre_latent_dim': 4,
    'pre_kappa': 10000,
    'pre_num_iterations': 2,
    'pre_fitting_desc': "initialization of model",
    'model_initialization': 'Yes',
} 

kpms_model.PreFittingTask.insert1(prefitting_key, skip_duplicates=True)


In [ ]:
# prefitting_keys = [{
#     **key,
#     'latent_dim': 3,
#     'kappa': int(i),
#     'num_iterations': 50,
#     'pre_fitting_desc': f"Prefitting {c}"
# } for c, i in enumerate(kappa_range, start=1)]

# prefitting_keys

In [ ]:
# for prefitting_key in prefitting_keys:
#     kpms_model.PreFittingTask.insert1(prefitting_key, skip_duplicates=True)

In [ ]:
kpms_model.PreFittingTask()

In [ ]:
kpms_model.PreFitting.populate()

In [ ]:
kpms_model.PreFitting()

In [ ]:
kpms_model.FullFittingTask()

In [ ]:
fullfitting_key = ({**key,
                      'full_latent_dim': 4,
                      'full_kappa': 10000,
                      'full_num_iterations':2,
                      'full_fitting_desc':"Testing full fitting model and generate results",
                      'task_mode':'trigger',
                      'sort_syllables':True,
                      'results_as_csv':True,
                      'visualizations':True})

kpms_model.FullFittingTask.insert1(fullfitting_key, skip_duplicates=True)                                  

In [ ]:
kpms_model.FullFittingTask()

In [ ]:
kpms_model.FullFitting.populate()

In [ ]:
kpms_model.FullFitting()

## 3. Generate results


In [ ]:
kpms_model.GenerateResults.populate()